# SOFTMAX REGRESSION

Softmax regression, also known as multinomial logistic regression, is a generalization of logistic regression to the case where we want to handle multiple classes. It's widely used for multiclass classification problems where each instance can belong to one of several classes. The goal of softmax regression is to estimate probabilities of each class that sum up to 1, making it a natural extension for problems beyond binary classification.


In Softmax Regression (also known as Multinomial Logistic Regression), the parameters (denoted as \(\theta\)) are updated using gradient descent or some other optimization technique. The update rule is derived from the cross-entropy loss function, which is used to measure the difference between the predicted probabilities and the actual labels.

### Softmax Function
First, the softmax function converts the linear combination of features into a probability distribution over classes:

$
P(y = j \mid x; \theta) = \frac{\exp(\theta_j^T x)}{\sum_{k=1}^{K} \exp(\theta_k^T x)}
$

where:
- $P(y = j \mid x; \theta)$ is the predicted probability that the input $x$ belongs to class $j$.
- $\theta_j$ is the parameter vector for class $j$.
- $K$ is the total number of classes.

### Cross-Entropy Loss Function
$J(\Theta) = -\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_k^{(i)} log(\hat{p}_k^{(i)})$

### Gradient Descent Update Rule - Vectorized Form
In vectorized form, for all classes $K$, the update rule is:

$
\Theta := \Theta - \alpha X^T (\mathbf{Y} - \mathbf{P})
$

Where:
- $\Theta$ is a matrix of parameters where each column corresponds to the parameter vector $\theta_j$ for each class $j$.
- $X$ is the matrix of input features.
- $\mathbf{Y}$ is a binary matrix indicating the actual classes for each example.
- $\mathbf{P}$ is the matrix of predicted probabilities for each class and each example.

This formula updates the parameter matrix $\Theta$ in the direction that reduces the cross-entropy loss, adjusting the model to improve its predictions.

# SOFTMAX REGRESSION FROM SCRATCH

In [83]:
import numpy as np
from my_extensions import MyRegressionBase
from sklearn.datasets import fetch_openml

# IMPORT MNIST DATA
mnist = fetch_openml("mnist_784", version=1)
X, y = np.array(mnist["data"]), np.array(mnist["target"])
np.random.seed(42)

In [84]:
X.shape

(70000, 784)

In [85]:
y.shape

(70000,)

In [86]:
y[:10]

array(['5', '0', '4', '1', '9', '2', '1', '3', '1', '4'], dtype=object)

In [87]:
# SPLIT DATA INTO TRAINING AND TESTING SETS

# Add 1s column to the X matrix to account for the bias term
X = np.c_[np.ones((len(X), 1)), X]

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]



In [88]:
# NORMALIZE DATA
X_train = X_train / 255
X_test = X_test / 255



In [89]:

# SOFTMAX FUNCTION
def softmax(logits):
    """
    Returns the values of the logits vector after applying the softmax function.
    Axis 1 is used to sum each row of the matrix and return new array. 
    Keep dims is used to keep the dimensions of the original array.
    [[1 2 3] [4 5 6]] -> [[6] [15]] 
    
    :param logits: Features, computed as linear model. Size: (nr_observations, n_classes)
    :return: np.array of probabilities where each row sums to 1. Size: (nr_observations, n_classes)
    """

    return np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)


# CROSS-ENTROPY LOSS FUNCTION               
def cross_entropy_loss(y, y_hat):
    """
    Compute the cross-entropy loss.

    Parameters:
    - y: np.array, one-hot encoded true labels. Shape: (nr_observations, n_classes)
    - y_hat: np.array, predicted probabilities. Shape: (nr_observations, n_classes)

    Returns:
    - loss: float, the cross-entropy loss averaged over all samples.
    """
    # Small constant to ensure numerical stability and avoid log(0)
    epsilon = 1e-7

    # Compute the cross-entropy loss
    loss = -np.sum(y * np.log(y_hat + epsilon))

    # Average over all samples
    loss /= y.shape[0]

    return loss


# CREATE SOFTMAX REGRESSION MODEL FROM SCRATCH
# Inherit from the Extension class to use its methods for visualization and evaluation
class MySoftmaxRegression(MyRegressionBase):
    def __init__(self, nr_of_features, nr_of_classes, learning_rate=0.1, n_iterations=1000):
        """
        Softmax regression model.
        
        It relies on linear regression model. All the computations are the same as in the Linear Regression model. 
        There are added few more things to it.
        In this case we have number of thetas as number of features (as usual) but also the same for each other class. 
        This in practise means that if you have in your dataPoints 3 features, and result could be one of 5 classes 
        then you will have 5 thetas x 3 features = 15 thetas, therefore thetas in a matrix shape (3, 5).
        
        In case of example in this notebook, we got 784 features (28x28 pixels) and 10 classes (digits from 0 to 9).
        The weight matrix will have shape (784, 10):
          class 0, class 1, ..., class 9
        [ w0_0   , w0_1   , ..., w0_9   ]
        [ w1_0   , w1_1   , ..., w1_9   ]
        ...
        [ w784_0 , w784_1 , ..., w784_9 ]
        
        The way I understand it is that we are basically preparing weights for all pixels and for each class.
        Therefore, if we then take this matrix and dot product it with features (using linear regression), we get a new 
        matrix where in each column we have scores/results corresponding to each class of a given datapoint. This means 
        that for example column 0 in the weight matrix (with 784 rows) will be calculated together with the feature 
        (all 784 pixels) and the resulting number/score is then the result for class 0 of a given datapoint. The same 
        goes for all other classes. 
        I noticed that the biggest number in each row is then latter also the winning class of each datapoint once the 
        softmax function is applied to it. (to the whole matrix)
        
        :param nr_of_features: Number of features per sample/observation 
        :param nr_of_classes: Number of possible classes per sample/observation
        :param learning_rate: How fast the model should learn
        :param n_iterations: How many times the model should learn
        """
        super().__init__()
        self.n_iterations = n_iterations
        self.learning_rate = learning_rate
        self.nr_features = nr_of_features
        self.nr_of_classes = nr_of_classes
        self.thetas = []
        
        # just for visualization ----------------------
        self.losses = []
        self.y_pred_linear = None
        self.y_pred_softmax = None
        self.gradients = None
        # ---------------------------------------------

    def fit(self, X, y):
        """
        Fit the model to the data X and y. Meaning let the model learn the parameters.
        :param X: Features
        :param y: Labels
        :return: None
        """

        nr_observations = len(X)
        
        # one-hot encode the labels (example in a cell below) Shape: (nr_observations, n_classes) (60_000, 10)
        Y = np.eye(self.nr_of_classes)[y.astype(int)]

        # initialize the parameters with random values Shape: (nr_features, n_classes) (785, 10)
        self.thetas = np.random.randn(self.nr_features, self.nr_of_classes)

        # gradient descent
        for iteration in range(self.n_iterations):
            # compute linear model (linear predictions) Shape: (nr_observations, n_classes) (60_000, 10)
            y_pred_linear = np.dot(X, self.thetas)

            # apply softmax function to get probabilities Shape: (nr_observations, n_classes) (60_000, 10)
            y_pred_softmax = softmax(y_pred_linear)
             
            # compute the gradients Shape: (nr_features, n_classes) (785, 10)
            gradients = 2 / nr_observations * np.dot(X.T, (y_pred_softmax - Y))

            # update the parameters Shape: (nr_features, n_classes) (785, 10)
            self.thetas = self.thetas - self.learning_rate * gradients
            
            # -----------------------------------------------------------------------
            # (only for visualization, not used in the optimization process)
            # compute the cost 
            loss = cross_entropy_loss(Y, y_pred_softmax)
            self.losses.append(loss)
            self.y_pred_linear = y_pred_linear
            self.y_pred_softmax = y_pred_softmax
            self.gradients = gradients
            
    def predict(self, X):
        """
        Predict the class labels for the provided data
        :param X: Features in the shape (nr_observations, nr_features) (any, 785)
        :return: Predicted class labels
        """
        linear_model = np.dot(X, self.thetas)
        return softmax(linear_model)


In [90]:
# 4 - we specify that we are working with 4 classes
# [0, 1, 2, 3, 3, 2, 1, 0] - Array we want to one-hot encode
Y = np.eye(4)[[0, 1, 2, 3, 3, 2, 1, 0]]
Y

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [91]:
# TRAIN THE MODEL (Takes 10 minutes to run. If you want to make it faster, then implement mini-batch gradient descent or SGD, instead of batch gradient descent)

model = MySoftmaxRegression(nr_of_features=X_train.shape[1], # 784, because 28x28 pixels + 1 for bias
                            nr_of_classes=len(np.unique(y_train)), # 10, because 10 digits
                            learning_rate=0.1,
                            n_iterations=10_000)
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# show the parameters
print("Best parameters:")
model.show_parameters()

In [ ]:
model.y_pred_linear

In [ ]:
rounded_softmax = model.y_pred_softmax.round(4)
rounded_softmax

In [ ]:
# there are places where are clean 0, I guess that might be due to the fact that the images have areas where the 
# pixels do not hold any information (the number which we predict is not there), so it just probably sets it to 0.
rounded_gradients = model.gradients.round(6)
rounded_gradients

In [ ]:
# VISUALIZE THE MODELS LOSS CURVE

model.show_loss_curve()

In [ ]:
# EVALUATE THE MODEL

y_pred = model.predict(X_test)

accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test.astype(int))
print(f"Accuracy: {accuracy:.2f}")

In those few cells below I demonstrate how you can take only one vector containing thetas/parameters of a given class and use only binary classificator

In [ ]:
# take a sample from the training set, which is number 0
X_sample_0 = X_train[108]

# visualise the sample with matplotlib
import matplotlib.pyplot as plt


def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

# [1:] to remove the bias term
plot_digit(X_sample_0[1:])

In [ ]:

def sigmoid(t):
    """
    Returns the values of the t vector after applying the sigmoid function.
    
    :param t: Feature vector. 
    :return: np.array
    """
    return 1 / (1 + np.exp(-t))

def predict(X, thetas):
    """
    Just a helper function to predict sigmoid class probabilities
    """
    linear_model = np.dot(X, thetas)
    y_predicted = sigmoid(linear_model)
    return y_predicted

for i, softmax_probability in enumerate(model.predict([X_sample_0])[0]):
    sigmoid_probability = predict(X_sample_0, model.thetas[:, i])
    print(f"Class:{i}", f"Sigmoid:{sigmoid_probability:.5f}", f"Softmax:{softmax_probability:.5f}", sep=", ")
